In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

In [ ]:
#define fontsize
fontsize = 17
figsize = (8,4)

# Configure parameters
plt.rcParams.update({'font.size': fontsize, 'figure.figsize': figsize})

# Set default tick and axes properties
plt.rcParams['xtick.labelsize'] = fontsize
plt.rcParams['ytick.labelsize'] = fontsize
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['xtick.major.width'] = 4
plt.rcParams['ytick.major.width'] = 4
plt.rcParams['xtick.minor.size'] = 6
plt.rcParams['ytick.minor.size'] = 6
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.minor.width'] = 2
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['axes.linewidth'] = 3

In [4]:
df_events1 = pd.read_csv('event-versions.csv') # load the event versions data with pandas
df_events2 = pd.read_csv('events.csv') # load the event data with pandas
df_events3 = pd.read_csv('/Users/leighharwell2/Desktop/G-Waves/events (1).csv') # corrected file path
df_events4 = pd.read_csv('/Users/leighharwell2/Desktop/G-Waves/events (2).csv')


In [7]:
df_events= pd.concat([df_events1, df_events2, df_events3, df_events4], ignore_index=True) # concatenate the dataframes
df_events

,name,shortName,gps,version,catalog,doi,detail_url,mass_1_source,mass_1_source_lower,mass_1_source_upper,...,redshift_upper,far,far_lower,far_upper,p_astro,p_astro_lower,p_astro_upper,final_mass_source,final_mass_source_lower,final_mass_source_upper
0,GW230529_181500,GW230529_181500-v1,1.369419e+09,1,O4_Discovery_Papers,https://doi.org/10.7935/6k89-7q62,http://gwosc.org/api/v2/event-versions/GW23052...,3.6,-1.2,0.8,...,0.02,0.00100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GW200322_091133,GW200322_091133-v1,1.268904e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,http://gwosc.org/api/v2/event-versions/GW20032...,38.0,-22.0,130.0,...,1.43,140.00000,NaN,NaN,0.61501,NaN,NaN,48.0,-22.0,132.0
2,GW200316_215756,GW200316_215756-v1,1.268431e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,http://gwosc.org/api/v2/event-versions/GW20031...,13.1,-2.9,10.2,...,0.08,0.00001,NaN,NaN,0.99000,NaN,NaN,20.2,-1.9,7.4
3,GW200311_115853,GW200311_115853-v1,1.267963e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,http://gwosc.org/api/v2/event-versions/GW20031...,34.2,-3.8,6.4,...,0.05,0.00001,NaN,NaN,0.99000,NaN,NaN,59.0,-3.9,4.8
4,GW200311_103121,200311_103121-v1,1.267958e+09,1,GWTC-3-marginal,https://doi.org/10.7935/b024-1886,http://gwosc.org/api/v2/event-versions/GW20031...,NaN,NaN,NaN,...,NaN,1.30000,NaN,NaN,0.19400,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,GW191126_115259,GW191126_115259-v1,1.258804e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,https://gwosc.org/api/v2/event-versions/GW1911...,12.1,-2.2,5.5,...,0.12,3.20000,NaN,NaN,0.70041,NaN,NaN,19.6,-2.0,3.5
268,GW191113_071753,GW191113_071753-v1,1.257665e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,https://gwosc.org/api/v2/event-versions/GW1911...,29.0,-14.0,12.0,...,0.18,26.00000,NaN,NaN,0.68339,NaN,NaN,34.0,-10.0,11.0
269,GW191109_010717,GW191109_010717-v1,1.257297e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,https://gwosc.org/api/v2/event-versions/GW1911...,65.0,-11.0,11.0,...,0.18,0.00018,NaN,NaN,0.99000,NaN,NaN,107.0,-15.0,18.0
270,GW191105_143521,GW191105_143521-v1,1.257000e+09,1,GWTC-3-confident,https://doi.org/10.7935/b024-1886,https://gwosc.org/api/v2/event-versions/GW1911...,10.7,-1.6,3.7,...,0.07,0.01200,NaN,NaN,0.99000,NaN,NaN,17.6,-1.2,2.1


In [8]:
def classify_merger(row):
    '''Classifiing merger types based on the 2 mass sources'''
    #define mass ranges
    NS_range = (1.1, 2.5) #neutron star mass range, 1.1-2.5 because some sources say 2 solar masses and others say 3 solar masses
    BH_range = 5 #balk Hole mass range is roughly 5 solar masses and up

    mass1, mass2 = row['mass_1_source'], row['mass_2_source']
    if NS_range[0] <= mass1 <= NS_range[1] and NS_range[0] <= mass2 <= NS_range[1]:
        return 'NS-NS'
    elif (NS_range[0] <= mass1 <= NS_range[1] and mass2 >= BH_range) or (NS_range[0] <= mass2 <= NS_range[1] and mass1 >= BH_range):
        return 'NS-BH'
    elif mass1 >= BH_range and mass2 >= BH_range:
        return 'BH-BH'
    else:
        return 'Unknown'

In [9]:
# Ensure the required columns 'mass_1_source' and 'mass_2_source' exist in the dataframe
if 'mass_1_source' in df_events.columns and 'mass_2_source' in df_events.columns:
    df_events['merger_type'] = df_events.apply(classify_merger, axis=1)
else:
    print("Required columns 'mass_1_source' and 'mass_2_source' are missing in the dataframe.")


In [14]:
columns_to_display = ['mass_1_source', 'mass_2_source','final_mass_source', 'merger_type']

# Filter the dataframe for NS-NS mergers
NS_NS_mergers = df_events[df_events['merger_type'] == 'NS-NS']

# DataFrame for NS-NS mergers
organized_df_2NS = NS_NS_mergers[columns_to_display]
print(organized_df_2NS)

     mass_1_source  mass_2_source  final_mass_source merger_type
125           1.74           1.56                NaN       NS-NS
126           2.00           1.40                NaN       NS-NS
127           2.10           1.30                NaN       NS-NS
147           1.46           1.27                2.8       NS-NS
184           1.46           1.27                2.8       NS-NS
230           2.10           1.30                NaN       NS-NS


In [15]:
# Filter the DataFrame for NS-BH mergers
NS_BH_mergers = df_events[df_events['merger_type'] == 'NS-BH']

# DataFrame for NS-BH mergers
organized_df_NS_BH = NS_BH_mergers[columns_to_display]
print(organized_df_NS_BH)

     mass_1_source  mass_2_source  final_mass_source merger_type
26             5.7           1.50                NaN       NS-BH
27             5.9           1.44                7.2       NS-BH
30             8.9           1.90                NaN       NS-BH
31             9.1           1.91               10.8       NS-BH
36            31.1           1.17               32.2       NS-BH
58             9.7           2.10               11.6       NS-BH
123            5.7           1.50                NaN       NS-BH
198            9.7           2.10               11.6       NS-BH
256            5.9           1.44                7.2       NS-BH
260           31.1           1.17               32.2       NS-BH


In [16]:
# Filter the DataFrame for BH-BH mergers
BH_BH_mergers = df_events[df_events['merger_type'] == 'BH-BH']

# DataFrame for BH-BH mergers
organized_df_BH_BH = BH_BH_mergers[columns_to_display]
print(organized_df_BH_BH)

     mass_1_source  mass_2_source  final_mass_source merger_type
1             38.0           11.3               48.0       BH-BH
2             13.1            7.8               20.2       BH-BH
3             34.2           27.7               59.0       BH-BH
5             60.0           24.0               88.0       BH-BH
6             28.3           14.8               41.7       BH-BH
..             ...            ...                ...         ...
267           12.1            8.3               19.6       BH-BH
268           29.0            5.9               34.0       BH-BH
269           65.0           47.0              107.0       BH-BH
270           10.7            7.7               17.6       BH-BH
271           11.8            7.9               19.0       BH-BH

[204 rows x 4 columns]
